# Practical Session 6: Introduction to Sensing

## Redes de Sensores y Sistemas Autónomos 
### Grado en Ingeniería de las Tecnologías de Telecomunicación
### Universidad de Sevilla

## Part 1: Laserscan message and Emergency stop node

To this point, our Turtlebot was a little bit sad and a feeling of loneliness embraced her as she explored an empty world. But this will change in this session. From now on, we will let our beloved Turtlebot explore a more interesting environment. Please start the simulation with:



If we want to avoid obstacles, we need to use a sensor to detect the obstacles. Particularly, our Turtlebot3 robot includes a 2D laser range finder: https://emanual.robotis.com/docs/en/platform/turtlebot3/appendix_lds_01/ . 
The Gazebo simulator is able to simulate this sensor among many others. Therefore, we can read the 2D laser scan readings, since our simulator is publishing this data. We can see with the tool rqt the publications. In this case we are interested in /scan topic [sensor_msgs/LaserScan].

From a 2D laser scan we obtain a set of distances (ranges) to the obstacles around the sensor. Each distance position in the vector of distances corresponds to a known angle, so we have a set of *polar coordinates* of the obstacles regarding the center of the sensor. 

Thus, we will subscribe to the topic /scan and receive this type of messages: sensor_msgs::LaserScan. The data structure is presented next:

In [ ]:
################ Contents of the LaserScan.msg file of package sensor_msgs ####################################
# Single scan from a planar laser range-finder
#
# If you have another ranging device with different behavior (e.g. a sonar
# array), please find or create a different message, since applications
# will make fairly laser-specific assumptions about this data

Header header            # timestamp in the header is the acquisition time of 
                         # the first ray in the scan.
                         #
                         # in frame frame_id, angles are measured around 
                         # the positive Z axis (counterclockwise, if Z is up)
                         # with zero angle being forward along the x axis
                         
float32 angle_min        # start angle of the scan [rad]
float32 angle_max        # end angle of the scan [rad]
float32 angle_increment  # angular distance between measurements [rad]

float32 time_increment   # time between measurements [seconds] - if your scanner
                         # is moving, this will be used in interpolating position
                         # of 3d points
float32 scan_time        # time between scans [seconds]

float32 range_min        # minimum range value [m]
float32 range_max        # maximum range value [m]

float32[] ranges         # range data [m] (Note: values < range_min or > range_max should be discarded)
float32[] intensities    # intensity data [device-specific units].  If your
                         # device does not provide intensities, please leave
                         # the array empty.


We will develop some scripts in a new ROS package, the `collision_avoidance` package. The idea of any node inside this package is that it susbscribes to the `v_pref` topic (preferred velocity), which usually comes from the path tracker node. Then, taking into account the obstacles sensed by the laser sensor, it generates a collision-free velocity as close as possible to the desired and publishes it in the `cmd_vel` topic, which is received and executed by the Turtlebot itself. This idea is depicted in Fig. 1.



__Exercise_1__

In a first approximation to the collision avoidance method, we will implement a basic node that subscribes the scan and stops the Turtlebot if an obstacle is detected with a distance minor than a given threshold `min_d`.

Thus, implement the node `emergency_stop.py` which subscribes to the `v_pref` and the `scan` topics. Then each time a scan is received it takes stop action if a threat is detected.

The node should periodically publish in the `cmd_vel` topic the last `v_pref` received under normal circumstances or zero velocity if a threat has been detected.